In [1]:
import torch as t
from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import visdom

input_dim = 10
n_hidden_1 = 32
n_hidden_2 = 16
n_hidden_21 = 8
n_hidden_22 = 8
n_hidden_3 = 8
n_hidden_31 = 4
n_hidden_32 = 4
n_hidden_4 = 16
output_dim = 2
output_dim1 = 1
output_dim2 = 1

vis = visdom.Visdom(env = u'face_angle')

In [68]:
#网络model
class Batch_Net(nn.Module):
    def __init__(self,input_dim,n_hidden_1,n_hidden_21,n_hidden_22,n_hidden_31,n_hidden_32,output_dim1,output_dim2):
        super().__init__()
        self.layer1 = nn.Sequential(
                                    nn.Linear(input_dim,n_hidden_1),
                                    nn.BatchNorm1d(n_hidden_1),
                                    nn.ReLU(True)
                                    )
        
        
        self.layer21 = nn.Sequential(
                                    nn.Linear(n_hidden_1,n_hidden_21),
                                    nn.BatchNorm1d(n_hidden_21),
                                    nn.ReLU(True)
                                    )
        self.layer22 = nn.Sequential(
                                    nn.Linear(n_hidden_1,n_hidden_22),
                                    nn.BatchNorm1d(n_hidden_22),
                                    nn.ReLU(True)
                                    )
        
        self.layer31 = nn.Sequential(
                                    nn.Linear(n_hidden_21,n_hidden_31),
                                    nn.BatchNorm1d(n_hidden_31),
                                    nn.ReLU(True)
                                    )
        self.layer32 = nn.Sequential(
                                    nn.Linear(n_hidden_22,n_hidden_32),
                                    nn.BatchNorm1d(n_hidden_32),
                                    nn.ReLU(True)
                                    )
        self.layer_out1 = nn.Sequential(
                                    nn.Linear(n_hidden_31,output_dim1),
                                    )
        self.layer_out2 = nn.Sequential(
                                    nn.Linear(n_hidden_32,output_dim2),
                                    )
        
    def forward(self,x):
        x = self.layer1(x)
        x21 = self.layer21(x)
        x22 = self.layer22(x)
        x31 = self.layer31(x21)
        x32 = self.layer32(x22)
        
        
        
        #x = self.layer4(x)      #想去掉某层，在这里就足够了。
        '''
        x1 = self.layer_out1(x)
        x2 = self.layer_out2(x)
        x = t.cat((x1,x2),1)           #这3行，即使和之前output_dim = 2效果是一样的，但是，说明可以使用cat。哈哈哈哈！
        '''
        x_out1 = self.layer_out1(x31)
        x_out2 = self.layer_out2(x32)
        x = t.cat((x_out1,x_out2),1)
        
        #print(x)
        return x

In [10]:
#网络model
class Batch_Net1(nn.Module):
    def __init__(self,input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim):
        super().__init__()
        self.layer1 = nn.Sequential(
                                    nn.Linear(input_dim,n_hidden_1),
                                    nn.BatchNorm1d(n_hidden_1),
                                    nn.ReLU(True)
                                    )
        self.layer2 = nn.Sequential(
                                    nn.Linear(n_hidden_1,n_hidden_2),
                                    nn.BatchNorm1d(n_hidden_2),
                                    nn.ReLU(True)
                                    )
        self.layer3 = nn.Sequential(
                                    nn.Linear(n_hidden_2,n_hidden_3),
                                    nn.BatchNorm1d(n_hidden_3),
                                    nn.ReLU(True)
                                    )
        self.layer_out = nn.Sequential(
                                    nn.Linear(n_hidden_3,output_dim),
                                    #nn.BatchNorm1d(output_dim),
                                    #nn.ReLU(True)
                                       )
        #self.softmax = nn.Softmax()
 
        
        #x = self.layer4(x)      #想去掉某层，在这里就足够了。
        '''
        x1 = self.layer_out1(x)
        x2 = self.layer_out2(x)
        x = t.cat((x1,x2),1)           #这3行，即使和之前output_dim = 2效果是一样的，但是，说明可以使用cat。哈哈哈哈！
      
        x_out1 = self.layer_out1(x31)
        x_out2 = self.layer_out2(x32)
        x = t.cat((x_out1,x_out2),1)
        '''
    def forward(self,x):
        #print("x.shape",x.shape)
        x = self.layer1(x)
        #print("x.shape1",x.shape)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer_out(x)
        #x = self.softmax(x)
        #print(x)
        return x

In [11]:
#dataset类
from torch.utils import data
import os
import numpy as np


class Feature_Points(data.Dataset):
    def __init__(self,root):
        all_folds = os.listdir(root)
        folds = [os.path.join(root,img) for img in all_folds]
        self.all_files = []
        for path in folds:
            one_files = os.listdir(path)
            files = [os.path.join(path,img) for img in one_files]
            self.all_files.extend(files) 
         
        
    def __getitem__(self,index):
        txt_path = self.all_files[index]
        label = 1 if 'front_face' in txt_path.split("\\")[-2] else 0
        fo = open(txt_path)
        data_str = fo.read()
        array = np.array(data_str.split(",")).astype(float)
        fo.close()
        data = t.from_numpy(array)
        
        return data, label
    
    def __len__(self):
        return len(self.all_files)        
  

In [18]:
batch_size = 64
learning_rate = 1e-4
num_epoches = 800
weight_decay = 1e-4

In [19]:
dataset = Feature_Points("D:\\newfolder7")
dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=True)
#data, label = dataset[5]
#data,label
#'''
dataiter = iter(dataloader)
datas,labels = next(dataiter)
datas,labels
#'''

(
  0.4612  0.4487  0.8223  0.5256  0.6335  0.7366  0.2834  0.7729  0.5736  0.8326
  0.2757  0.3660  0.7407  0.3447  0.4925  0.6085  0.3409  0.7983  0.7258  0.7779
  0.3154  0.3644  0.7716  0.3448  0.5968  0.5794  0.3677  0.7608  0.7734  0.7483
  0.3020  0.3954  0.7400  0.4733  0.4417  0.5995  0.2189  0.7325  0.6008  0.7954
  0.6492  0.4091  0.8906  0.4500  0.8521  0.7079  0.4967  0.7656  0.7439  0.8241
  0.3272  0.3831  0.8061  0.3215  0.5980  0.6500  0.3644  0.7529  0.8145  0.7021
  0.5609  0.3310  0.8972  0.3852  0.7833  0.5982  0.3953  0.7613  0.6212  0.8020
  0.2153  0.3471  0.6643  0.3439  0.3688  0.5385  0.2619  0.7663  0.6070  0.7629
  0.2923  0.3972  0.7752  0.4222  0.4887  0.6263  0.2753  0.7673  0.7036  0.7842
  0.3414  0.4072  0.7887  0.3970  0.6202  0.6090  0.4265  0.7886  0.7428  0.7908
  0.1614  0.3766  0.5068  0.3869  0.1538  0.5960  0.1498  0.7886  0.4369  0.7875
  0.4440  0.3656  0.8876  0.3904  0.7282  0.6389  0.4161  0.7518  0.7791  0.7728
  0.1866  0.3707  0.3984  

In [20]:
#model = Batch_Net(input_dim,n_hidden_1,n_hidden_21,n_hidden_22,n_hidden_31,n_hidden_32,output_dim1,output_dim2)
model = Batch_Net1(input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim)

In [21]:
if t.cuda.is_available():
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
criterion.cuda()   #P161
optimizer = optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay)  

#loss_meter = meter.AverageValueMeter()

In [22]:
#参考的siamese网络的那个结构
sum_loss = t.Tensor([0])     #多一个0元素，不会影响平均值
for epoch in range(0,num_epoches):
    for i,data in enumerate(dataloader,0):   #表示i 从0开始
        data_batch, label = data
        data_batch, label = Variable(data_batch.float()).cuda(),Variable(label).cuda()
        output = model(data_batch)
        #print(output.cpu())
        #print(output)
        optimizer.zero_grad()
        
        loss = criterion(output,label)
        loss.backward()            #我终于知道，我的图为什么不下降了，我忘记反向传播了。。。。。。。。。。。。
        optimizer.step()
        #sum_loss.append(loss)
        sum_loss = t.cat((sum_loss,loss.cpu().data),0)
        x = i+epoch*len(dataloader)      #此时的x还只是个常数，不是列表，也不是np.array()
        
        if x%50 == 49:
            loss_aver = sum_loss.mean()
            sum_loss = t.Tensor([0])
            #vis.line(X=x, Y=loss.cpu().data, win='face_angle',opts={'title': 'loss-batch', 'xlabel':'x','ylabel':'y'},update='append' if i>=0 else None)
            vis.line(X=t.Tensor([x]), Y=t.Tensor([loss_aver]), win='face_angle1',opts={'title': 'loss-batch1', 'xlabel':'batch_num','ylabel':'loss_aver'},update='append' if x>49 else None)
            #vis.updateTrace(X=t.Tensor([x]),Y=t.Tensor([loss_aver]),win='face_angle', name='7')
t.save(model.state_dict(),"C:\\Users\\SiChen\\face_sys\\FC_network\\model_save\\fc_network_800_32_16_NoSoftmax_lr1e-5.pth")        

In [9]:
#实验，t.cat在这里有没有用
sum_loss = t.cat((sum_loss,loss.cpu().data),0)
#sum_loss

In [101]:
#通过本例，可以看出list array tensor三者的异同
#所以可以对tensor、numpy.array使用sum()函数。但是对list却不可以
#在test的统计中，使用的会比较多。

a = [1,0,1]
b = [1,1,1]
c = (a==b)
print(c)

a0 = np.array(a)
b0 = np.array(b)
c0 = (a0==b0)
print(c0.sum())  #c0.sum()是numpy.int32
print(c0)

aa = t.Tensor(a)
bb = t.Tensor(b)
cc = (aa==bb)
print(cc.sum())   #cc.sum()是int类型

a1 = Variable(aa)
b1 = Variable(bb)
c1 = (a1==b1)

print(c1.sum())   #c1.sum()是torch.ByteTensor。size为1

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!这几个类型，这次可以非常清楚它们的差别了。知道在jupyter中如何区分了!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
print("c1",c1)     #类型为：Variable containing: [torch.ByteTensor of size 3]       是Variable.
print(c1.data)     #类型为：[torch.ByteTensor of size 3]  不含Variable containing:。即，不是Variable。只是单纯的Tensor
print("c1.data[0]",c1.data[0])  #类型为int


for i in c1:
    print(i)
v= Variable(t.Tensor([1])).type(t.ByteTensor)
c1[c1==0]
c1[1]

a0[1] = 5
a0[1]
#c1[c1==0]

#c1.data[c1.data==0] = t.Tensor([1]).type(t.ByteTensor)      #Variable不知道为什么不可以修改c1[1]，但是numpy.array，list，Tensor都可以。所以通过这种方式。
#c1.data[c1.data>0] = t.zeros(2).type(t.ByteTensor)
print("before process c1[c1>0]   ",c1[c1>0])
##============================================================================###
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!通过这段代码，可以将所有的0变成1，所有的1变成0。非常好用啊！！！！！！！！！！！！！！！！！！！！！
#此处的c1是Variable.
t1 = (c1.data==0)
t2 = (c1.data>0)
print("t1",t1)
print("t2",t2)
print(t1.sum())
print(t2.sum())
print("c1.data[t1]",c1.data[t1])
print("c1.data[t2]",c1.data[t2])
c1.data[t1] = t.ones(t1.sum()).type(t.ByteTensor)         
c1.data[t2] = t.zeros(t2.sum()).type(t.ByteTensor)
print("after process c1[c1>0]   ",c1[c1>0])
###=============================================================================###

False
2
[ True False  True]
2
Variable containing:
 2
[torch.ByteTensor of size 1]

c1 Variable containing:
 1
 0
 1
[torch.ByteTensor of size 3]


 1
 0
 1
[torch.ByteTensor of size 3]

c1.data[0] 1
Variable containing:
 1
[torch.ByteTensor of size 1]

Variable containing:
 0
[torch.ByteTensor of size 1]

Variable containing:
 1
[torch.ByteTensor of size 1]

before process c1[c1>0]    Variable containing:
 1
 1
[torch.ByteTensor of size 2]

t1 
 0
 1
 0
[torch.ByteTensor of size 3]

t2 
 1
 0
 1
[torch.ByteTensor of size 3]

1
2
c1.data[t1] 
 0
[torch.ByteTensor of size 1]

c1.data[t2] 
 1
 1
[torch.ByteTensor of size 2]

after process c1[c1>0]    Variable containing:
 1
[torch.ByteTensor of size 1]



# 保存模型与加载模型

In [23]:
t.save(model.state_dict(),"C:\\Users\\SiChen\\face_sys\\FC_network\\model_save\\fc_network_800_32_16_NoSoftmax_lr1e-4.pth")

In [84]:
#加载模型必须在，前面定义模型之后，即在前面的model = ....之后才能执行这一行。
model.load_state_dict(t.load("C:\\Users\\SiChen\\face_sys\\FC_network\\model_save\\fc_network_800_32_16_softmax.pth"))

# 以下是测试部分

In [24]:
test_dataset = Feature_Points("D:\\newfolder17")
test_dataloader = DataLoader(test_dataset,batch_size=3,shuffle=True)

In [26]:
model.cuda()
model.eval()
test_right_num = 0
for data in test_dataloader:
    data_batch,label = data
    print(data_batch.shape)
    data_batch,label = Variable(data_batch.float(),volatile=True).cuda(),Variable(label,volatile=True).cuda()
    out = model(data_batch)
    
    _,pred = t.max(out.cpu(),1)
    num_correct = (pred==label.cpu()).sum()
    test_right_num += num_correct
    
    qq = (pred==label.cpu())
    t1 = (qq.data==0)
    t2 = (qq.data>0)
    qq.data[t1] = t.ones(t1.sum()).type(t.ByteTensor)         
    qq.data[t2] = t.zeros(t2.sum()).type(t.ByteTensor)
    #print("after process c1[c1>0]   ",t.Tensor(data)([qq>0]))#????????????????????????
    
acc = (float(test_right_num))/len(test_dataset)
print(test_right_num)
print(len(test_dataset))
print(acc)

torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([3, 10])
Variable containing:
 103
[torch.ByteTensor of size 1]

105
0.9809523809523809


In [40]:
print(len(os.listdir("D:\\newfolder7\\front_face")))
(len(os.listdir("D:\\newfolder7\\side_face")))

3754


1476

In [42]:
print(len(os.listdir("D:\\newfolder17\\front_face")))
(len(os.listdir("D:\\newfolder17\\side_face")))

44


61

In [2]:
import numpy as np

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.array([[9,8,7],[6,5,4],[3,2,1]])
c = np.maximum(a,b)
a1 = np.array([[1,2,3],[4,5,6],[7,8,9]])
b1 = np.array([[9,8,7],[6,5,4],[3,2,1]])
c1 = np.minimum(a,b)
d = c - c1
print(d[1].var())
print(d[2].var())
print(d[0].var())
(d[0].var()+d[1].var()+d[2].var())/3
d.var()

0.888888888888889
2.6666666666666665
2.6666666666666665


6.913580246913581